<a href="https://colab.research.google.com/github/KennyThinh/pytorch/blob/main/03_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as f #function without parameter relu, tank
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms #transformation to perform on dataset

In [29]:
# create fully connected network
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes=10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) #output size = input_size
    self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))    
    self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.fc = nn.Linear(in_features= 16*7*7, out_features = num_classes, bias=True)
  
  def forward(self, x):
    # print("input:", x.shape)
    x = f.relu(self.conv1(x))
    # print("conv1:", x.shape)
    x = self.pool1(x)
    # print("pool1:", x.shape)
    x = f.relu(self.conv2(x))
    # print("conv2:", x.shape)
    x = self.pool2(x)    
    # print("pool2:", x.shape)
    x = x.reshape(x.shape[0], -1)
    # print("reshape:", x.shape)
    x = self.fc(x)
    # print("fc:", x.shape)
    return x

#test
model = CNN()
x = torch.rand(64,1,28,28)
print(model(x).shape)

# input: torch.Size([64, 1, 28, 28])
# conv1: torch.Size([64, 8, 28, 28])
# pool1: torch.Size([64, 8, 14, 14])
# conv2: torch.Size([64, 8, 14, 14])
# pool2: torch.Size([64, 8, 7, 7])
# reshape: torch.Size([64, 392])
# fc: torch.Size([64, 10])
# torch.Size([64, 10])

torch.Size([64, 10])


In [30]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# hyper parameter
# input_size = 784
num_classes = 10
num_epochs = 1
batch_size = 64
learning_rate = 0.001

In [32]:
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz -P dataset/
# !tar -zxvf dataset/MNIST.tar.gz -C dataset/

train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root="dataset/", train = False, transform=transforms.ToTensor(), download=False)
test_loader = DataLoader(test_dataset, batch_size= batch_size, shuffle = True)

In [33]:

#Init the network
model = CNN(in_channels=1, num_classes=10).to(device)

In [34]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
#train 
for epoch in range(num_epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device)
    target = target.to(device)
    
    #forward
    scores = model(data)
    loss = criterion(scores, target)

    #backward
    optimizer.zero_grad() #to clear out gradients of last time
    loss.backward()

    #gradient to update params
    optimizer.step() 

In [36]:
#check acc on train and test
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval() #inform that we are going to evaluate
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)

      # x = x.reshape(x.shape[0], -1)
      scores = model(x) #dim (64,10)      
      _, predictions = torch.max(scores, dim=1) #find the max among 10 outputs, so the dim is 1
      #predictions ([64])
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0) #each time is 64 examples
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Got 57338/60000 with accuracy 95.56
Got 9585/10000 with accuracy 95.85
